In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

In [2]:
dataset = pd.read_csv("Stocks_train.csv")

In [3]:
dataset

,Date,Stock,Close
0,2025-05-30,ASHOKLEY,236.00
1,2025-05-29,ASHOKLEY,240.65
2,2025-05-28,ASHOKLEY,238.50
3,2025-05-27,ASHOKLEY,239.55
4,2025-05-26,ASHOKLEY,239.85
...,...,...,...
3680,2022-06-07,SBIN,463.35
3681,2022-06-06,SBIN,463.55
3682,2022-06-03,SBIN,464.30
3683,2022-06-02,SBIN,469.75


In [4]:
# Pivot with stock column
dataset_train = dataset.pivot(index='Date',columns='Stock',values='Close').reset_index()
dataset_train

Stock,Date,ASHOKLEY,CANBK,LICI,ONGC,SBIN
0,2022-06-01,140.00,208.65,810.55,149.70,468.20
1,2022-06-02,143.45,213.50,805.85,152.50,469.75
2,2022-06-03,139.60,211.10,800.25,151.55,464.30
3,2022-06-06,138.35,210.30,777.40,154.00,463.55
4,2022-06-07,139.60,209.10,752.90,161.85,463.35
...,...,...,...,...,...,...
732,2025-05-26,239.85,107.45,870.00,245.80,794.35
733,2025-05-27,239.55,108.44,871.05,244.55,793.80
734,2025-05-28,238.50,110.88,942.55,242.70,796.45
735,2025-05-29,240.65,110.81,944.40,243.35,797.30


In [5]:
# Feature Scaling
num_cols = ['ASHOKLEY','CANBK','LICI','ONGC','SBIN']
sc = MinMaxScaler(feature_range=(0,1))
train_data = sc.fit_transform(dataset_train[num_cols])

In [6]:
train_data

array([[0.07674858, 0.23314899, 0.42471396, 0.13020833, 0.07199661],
       [0.10283554, 0.24200113, 0.41754386, 0.14288949, 0.07528364],
       [0.07372401, 0.23762069, 0.40900076, 0.13858696, 0.06372601],
       ...,
       [0.82155009, 0.05470076, 0.62608696, 0.55140399, 0.76810519],
       [0.83780718, 0.054573  , 0.62890923, 0.55434783, 0.76990775],
       [0.8026465 , 0.06189199, 0.64286804, 0.53668478, 0.80192981]])

In [7]:
# Creating a data structure with 60 time steps and 1 output
X_train = []
y_train = []
for i in range(60,len(train_data)):
    X_train.append(train_data[i-60:i,:])
    y_train.append(train_data[i,:])
X_train, y_train = np.array(X_train), np.array(y_train)

In [8]:
X_train.shape

(677, 60, 5)

In [54]:
regressor = Sequential()

regressor.add(GRU(units=200,activation='relu',return_sequences=True,input_shape=(X_train.shape[1],X_train.shape[2])))
regressor.add(Dropout(0.2))

"""regressor.add(GRU(units=118,activation='relu', return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(GRU(units=118,activation='relu', return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(GRU(units=118,activation='relu', return_sequences=True))
regressor.add(Dropout(0.2))"""

regressor.add(GRU(units=200,activation='relu'))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=X_train.shape[2]))

regressor.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [55]:
# printing the summary of RNN
print(regressor.summary())

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_31 (GRU)                         │ (None, 60, 200)             │         124,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_31 (Dropout)                 │ (None, 60, 200)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_32 (GRU)                         │ (None, 200)                 │         241,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_32 (Dropout)                 │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 5)                   │           1,005 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 366,405 (1.40 MB)

 Trainable params: 366,405 (1.40 MB)

 Non-trainable params: 0 (0.00 B)

None


In [56]:
early_stopping = EarlyStopping(monitor='val_accuracy',patience=5)

In [57]:
history = regressor.fit(X_train,y_train,epochs=25, batch_size=70,validation_split=0.2)

Epoch 1/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 526ms/step - accuracy: 0.1479 - loss: 0.2033 - val_accuracy: 0.2206 - val_loss: 0.0573
Epoch 2/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 386ms/step - accuracy: 0.4655 - loss: 0.0399 - val_accuracy: 0.2721 - val_loss: 0.0274
Epoch 3/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 388ms/step - accuracy: 0.5750 - loss: 0.0228 - val_accuracy: 0.2206 - val_loss: 0.0087
Epoch 4/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 391ms/step - accuracy: 0.8116 - loss: 0.0119 - val_accuracy: 0.2941 - val_loss: 0.0084
Epoch 5/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 387ms/step - accuracy: 0.8176 - loss: 0.0105 - val_accuracy: 0.7794 - val_loss: 0.0075
Epoch 6/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 388ms/step - accuracy: 0.7995 - loss: 0.0083 - val_accuracy: 0.8235 - val_loss: 0.0050
Epoch 7/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 391ms/step - accuracy: 0.8540 - loss: 0.0074 - val_accuracy: 0.4412 - val_loss: 0.0067
Epoch 8/25
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 387ms/step - accuracy: 0.8371 - loss: 0.0078 - val_accuracy: 0.4706 - val_loss